[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/11_Advanced_Context_Engineering.ipynb)

# Advanced Context Engineering

## Overview

This notebook covers advanced topics in context engineering using Semantica. We will explore custom memory management strategies, tuning hybrid retrieval, and extending the system with custom graph builders.

### Learning Objectives

- **Custom Memory Pruning**: Implement importance-based pruning instead of standard FIFO/token-based pruning.
- **Custom Graph Extensions**: Register custom graph building methods using the registry system.
- **Hybrid Retrieval Tuning**: Optimize weights for vector and graph search.

---

## 1. Setup

We'll start by setting up the environment and initializing a standard Vector Store.

In [1]:
!pip install -q semantica

In [2]:
import logging
from typing import List, Dict, Any, Optional
from semantica.context import AgentMemory, AgentContext, ContextGraph, ContextRetriever
from semantica.vector_store import VectorStore
from semantica.context import registry, methods

# Configure logging to see internal processes
logging.basicConfig(level=logging.INFO)

# Initialize Vector Store (using in-memory backend for this example)
# In production, you might use 'weaviate', 'qdrant', or 'faiss'
vs = VectorStore(backend="inmemory", dimension=384)

# Initialize Context Graph
kg = ContextGraph()

c:\Users\Mohd Kaif\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO:semantica.embedding_generator:Embedding generator initialized


## 2. Custom Memory Pruning Strategy

By default, `AgentMemory` uses a FIFO (First-In-First-Out) strategy combined with a token limit to prune short-term memory. However, you might want to keep "important" memories longer regardless of their age.

Let's subclass `AgentMemory` to implement an importance-based pruning strategy that respects metadata flags.

In [3]:
class ImportanceAwareMemory(AgentMemory):
    def _prune_short_term_memory(self) -> None:
        """
        Custom pruning: Always keep items marked as 'important' in metadata,
        then prune others based on token limits.
        """
        if not self.short_term_memory:
            return

        # Separate important items
        important_items = [item for item in self.short_term_memory if item.metadata.get("important")]
        other_items = [item for item in self.short_term_memory if not item.metadata.get("important")]
        
        # Calculate tokens used by important items
        important_tokens = sum(self._count_tokens(item.content) for item in important_items)
        
        # Calculate remaining budget
        remaining_tokens = max(0, self.token_limit - important_tokens)
        
        # Prune other items to fit remaining budget
        kept_others = []
        current_tokens = 0
        
        # Iterate in reverse (newest first) to keep recent items
        for item in reversed(other_items):
            item_tokens = self._count_tokens(item.content)
            if current_tokens + item_tokens <= remaining_tokens:
                kept_others.insert(0, item)
                current_tokens += item_tokens
            else:
                break # Stop once we hit the limit
                
        # Reconstruct memory: Important items + kept recent items
        # Sort by timestamp to maintain order
        all_kept = sorted(important_items + kept_others, key=lambda x: x.timestamp)
        self.short_term_memory = all_kept

# Initialize our custom memory with a strict token limit for testing
memory = ImportanceAwareMemory(
    vector_store=vs, 
    token_limit=100, 
    short_term_limit=50
)

# 1. Store an OLD but IMPORTANT memory
memory.store("IMPORTANT: User's name is Alice", metadata={"important": True})

# 2. Flood memory with newer filler content
for i in range(20):
    memory.store(f"Filler memory {i} " * 5) # This consumes tokens

print(f"Short-term items count: {len(memory.short_term_memory)}")
print("First item (should be the important one):", memory.short_term_memory[0].content)

Status,Action,Module,Submodule,File,Time
✅,Semantica is processing,🔗 context,AgentMemory,-,0.03s
✅,Semantica is embedding,💾 embeddings,TextEmbedder,-,0.01s
✅,Semantica is indexing,📊 vector_store,VectorStore,-,0.00s
✅,Semantica is processing,🔗 context,ContextRetriever,-,0.04s


INFO:semantica.progress:[RUNNING] | Module: context | Submodule: AgentMemory | Message: Storing memory: IMPORTANT: User's name is Alice...
INFO:semantica.progress:[RUNNING] | Module: context | Submodule: AgentMemory | Message: Generating embedding...
INFO:semantica.progress:[RUNNING] | Module: embeddings | Submodule: TextEmbedder | Message: Generating text embedding: IMPORTANT: User's name is Alice...
INFO:semantica.progress:[RUNNING] | Module: embeddings | Submodule: TextEmbedder | Message: Using fallback embedding method...
INFO:semantica.progress:[COMPLETED] | Module: embeddings | Submodule: TextEmbedder | Message: Generated embedding (dim: 16)
INFO:semantica.progress:[RUNNING] | Module: vector_store | Submodule: VectorStore | Message: Storing 1 vectors
INFO:semantica.progress:[RUNNING] | Module: vector_store | Submodule: VectorStore | Message: Storing vectors...
INFO:semantica.progress:[RUNNING] | Module: vector_store | Submodule: VectorStore | Message: Updating vector index...
INF

Short-term items count: 5
First item (should be the important one): IMPORTANT: User's name is Alice


## 3. Extending with Custom Graph Methods

Semantica's registry system allows you to plug in custom logic for graph construction, retrieval, and more. This is powerful for domain-specific graph topologies.

Let's register a custom graph builder that creates a "Star Graph" topology.

In [4]:
def star_graph_builder(
    entities: Optional[List[Dict[str, Any]]] = None,
    relationships: Optional[List[Dict[str, Any]]] = None,
    conversations: Optional[List[Any]] = None,
    center_entity: str = "Central Hub",
    satellites: Optional[List[str]] = None,
    **kwargs
) -> Dict[str, Any]:
    """
    Builds a star graph where all satellites connect to the center.
    """
    nodes = []
    edges = []
    
    # Center node
    nodes.append({"id": "center", "type": "CENTER", "properties": {"content": center_entity}})
    
    satellites = satellites or []
    for i, sat in enumerate(satellites):
        sat_id = f"sat_{i}"
        nodes.append({"id": sat_id, "type": "SATELLITE", "properties": {"content": sat}})
        edges.append({"source_id": "center", "target_id": sat_id, "type": "connects_to"})
        
    return {
        "nodes": nodes, 
        "edges": edges, 
        "statistics": {"node_count": len(nodes), "edge_count": len(edges)}
    }

# Register the method in the global registry
registry.method_registry.register("graph", "star_builder", star_graph_builder)

# Verify registration
print("Available graph methods:", registry.method_registry.list_all("graph"))

Available graph methods: {'graph': ['entities_relationships', 'conversations', 'hybrid', 'star_builder']}


Now we can use this method via the standard `methods` interface.

In [5]:
# Build a graph using our custom method
graph_data = methods.build_context_graph(
    method="star_builder",
    center_entity="Solar System",
    satellites=["Earth", "Mars", "Jupiter", "Venus"]
)

print(f"Created graph with {len(graph_data['nodes'])} nodes and {len(graph_data['edges'])} edges.")
print("Edges sample:", graph_data['edges'][0])

Created graph with 5 nodes and 4 edges.
Edges sample: {'source_id': 'center', 'target_id': 'sat_0', 'type': 'connects_to'}


## 4. Tuning Hybrid Retrieval

Hybrid retrieval combines scores from vector search and graph traversal. You can tune the `hybrid_alpha` parameter to weight these components.

- `hybrid_alpha = 0.0`: Pure Vector Search
- `hybrid_alpha = 1.0`: Pure Graph Search
- `hybrid_alpha = 0.5`: Balanced (Default)

Let's configure a `ContextRetriever` with a preference for graph connections.

In [6]:
# Populate knowledge graph with some test data
kg.add_node("python", "concept", "Python")
kg.add_node("ml", "concept", "Machine Learning")
kg.add_edge("python", "ml", "used_for")

# Initialize retriever with custom tuning
retriever = ContextRetriever(
    memory_store=memory,
    knowledge_graph=kg,
    vector_store=vs,
    hybrid_alpha=0.7,      # Favor graph connections
    max_expansion_hops=2   # Traverse deeper in the graph
)

# Retrieve
results = retriever.retrieve("Python")

print(f"Found {len(results)} results.")
for res in results:
    print(f"Source: {res.source}, Score: {res.score:.2f}, Content: {res.content[:50]}...")

INFO:semantica.progress:[RUNNING] | Module: context | Submodule: ContextRetriever | Message: Retrieving context for: Python...
INFO:semantica.progress:[RUNNING] | Module: context | Submodule: ContextRetriever | Message: Retrieving from vector store...
INFO:semantica.progress:[RUNNING] | Module: context | Submodule: ContextRetriever | Message: Retrieving from knowledge graph...
INFO:semantica.progress:[RUNNING] | Module: context | Submodule: ContextRetriever | Message: Retrieving from memory...
INFO:semantica.progress:[RUNNING] | Module: context | Submodule: AgentMemory | Message: Retrieving memories for: Python...
INFO:semantica.progress:[RUNNING] | Module: context | Submodule: AgentMemory | Message: Searching vector store...
INFO:semantica.progress:[RUNNING] | Module: embeddings | Submodule: TextEmbedder | Message: Generating text embedding: Python...
INFO:semantica.progress:[RUNNING] | Module: embeddings | Submodule: TextEmbedder | Message: Using fallback embedding method...
INFO:sem

Found 1 results.
Source: graph:python, Score: 1.00, Content: Python...


## Conclusion

You have successfully extended Semantica's context capabilities by:
1.  Implementing a custom memory pruning logic.
2.  Registering a new graph construction algorithm.
3.  Tuning the hybrid retrieval parameters.

These patterns allow you to adapt the context engine to specialized domain requirements.